<a href="https://colab.research.google.com/github/monikamikhaiel/graduation-project-/blob/main/edgeDetection_heat_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# read the image
image = cv2.imread("/content/master_bathroom_floor_plan_3.jpg")
# convert it to grayscale
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# show the grayscale image
#plt.imshow(gray, cmap="gray")
#plt.show()
# perform the canny edge detector to detect image edges
edges = cv2.Canny(image,threshold1=30, threshold2=100)
print(edges)
plt.imshow(edges)
plt.show()
cv2.destroyAllWindows()
#cv2.waitkey(0)
xcoord=[]
ycoord=[]
width,length,_=image.shape

In [ ]:
## detect the walls 
TupleWallsIndex=[]
for i in range (width):
  for j in range (length):
    if edges[i][j] == 255:
      TupleWallsIndex.append((i,j))
print(TupleWallsIndex)
SIZE_H=length
SIZE_W=width

In [ ]:
#antenna
class antenna:
    def __init__(self,x_,y_):
      if not x_:
        self.x = np.random.randint(0, SIZE_W)
      else:
        self.x=x_
      if not y_:
        self.y = np.random.randint(0, SIZE_H)
      else:
        self.y=y_
    def __str__(self):
        return f"{self.x}, {self.y}"
    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)
    def action(self, choice):
        '''
        Gives us 4 total movement options. (0,1,2,3)
        '''
        if choice == 0:  #up
            #self.move(x=0, y=1)
            self.x=self.x
            self.y+=1
            self.move()

        elif choice == 1:  #down
            #self.move(x=0, y=-1)
            self.x=self.x
            self.y-=1
            self.move()

        elif choice == 2:  #left
           # self.move(x=-1, y=0)
            self.x-=1
            self.y=self.y
            self.move()

        elif choice == 3:    #right
            # self.move(x=1, y=0)
            self.x+=1
            self.y=self.y
            self.move()

#    def move(self, x=False, y=False):
    def move(self):
        # If no value for x, move randomly
      #  if not x:
      #      self.x += np.random.randint(-1, 2)
       # elif x==0:
      #      self.x=self.x
      #  else:
       #     self.x += x

        # If no value for y, move randomly
       # if not y:
        #    self.y += np.random.randint(-1, 2)
       # elif y==0:
        #    self.y=self.y
       # else:
        #    self.y += y
      # If we are out of bounds, fix!
        if self.x < 0:
            self.x = 0
        elif self.x > SIZE_W-1:
            self.x = SIZE_W-1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE_H-1:
            self.y = SIZE_H-1

ant=antenna(5,6)

In [ ]:
def dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W):
  dict_values_edges={}
  for i in range(SIZE_H):
    for j in range(SIZE_W):
      for n in range(len(TupleWallsIndex)):
        if n<len(TupleWallsIndex)-1:
          x,y=TupleWallsIndex[n]
          x1,y1=TupleWallsIndex[n+1]
       # print(x,i,x1,y,j,y1)
          if x<=i<=x1:
            if y<=j<=y1:
              #print(x,i,x1,y,j,y1)
              dict_values_edges[i,j]=10
          else:
              dict_values_edges[i,j]=1
        else:
          break
  return dict_values_edges 
      
dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)
print(len(dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)))
print(dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)[0,1])



In [ ]:
#okomora model 
dict_values_edges=dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)
def okomora(ant,SIZE_H,SIZE_W,dict_values_edges=dict_values_edges):
  PathLoss=[]
  attenuation= 13 #in db function of frequency and distance 
  freq= 2.4*(10^9) # the frequency og the signal
  c= 3*(10^8) # the speed of light
  #hte = input("please enter the height of the transmitter antenna ? ")
  #hre = input("please enter the height of the receiver antenna ? ")
  hte= 20
  hre= 2
  G_hte = 10*np.log(int(hte) /200) 
  G_hre = 10*np.log(int(hre)/ 3)
  G_Area =27 # (Quasi Open Area)
  for i in range(SIZE_H):
    for j in range(SIZE_W):
      difference = [ant.x-i,ant.y-j]
      if difference == [0,0]:
        continue
      else:
        ## calculate the pathloss
        distance=pow((pow(difference[0],2)+pow(difference[1],2)),0.5)
        #values=dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)
        if (i,j) in dict_values_edges.keys():
          x=dict_values_edges[i,j]
          distance*=x
        FPL      = 20*np.log(4*np.pi*distance*freq/c)
        Path_Loss = FPL + attenuation - G_hte - G_hre - G_Area
          #Path_Loss*=10
        PathLoss.append(Path_Loss)
  return PathLoss  
#okomora(ant,SIZE_H,SIZE_W)  
#len(okomora(ant,SIZE_H,SIZE_W))

In [ ]:
def AnglesInDegrees(ant,SIZE_H,SIZE_W):
  theta=[]
  for i in range(SIZE_H):
    for j in range(SIZE_W):
      difference = [ant.x-i,ant.y-j]
      if difference==[0,0]: # the same position as the antenna 
        continue
      if difference[0]==0:     # the angle itself is either 90 or 270 tan will be infinite 
          if difference[1]>0:
            t=90
            theta.append(90)
          else:
            t=270
            theta.append(270)
      else:
          tantheta= difference[1]/difference[0]
          t=np.degrees(np.arctan(tantheta))
          if t <=0:
            t+=360
            theta.append(t)
          else:
            theta.append(t)
  return theta
#print(len(AnglesInDegrees(ant,SIZE_H,SIZE_W)))

In [ ]:
def measureSNR(ant,SIZE_H,SIZE_W,typeOfAntenna="omnidirectionnel",dict_values_edges=dict_values_edges):
  Path_Loss=okomora(ant,SIZE_H,SIZE_W,dict_values_edges)
  Gain=[]
  if typeOfAntenna == "omnidirectionnel" :
    Gain_pannel=7 #in db
    for i in range(len(Path_Loss)):
      Gain.append(Gain_pannel-Path_Loss[i])
  elif typeOfAntenna == "directionnel":
    thetas=AnglesInDegrees(ant,SIZE_H,SIZE_W)
    GainVsTheta={360:-20,30:-10,60:-5,90:15,120:0,150:-10,180:-20,210:-10,240:-8,270:-15,300:-8,330:-10} # values in db angle:gain
    for i in range(len(thetas)):
      t=thetas[i]
      if t==0:
       value = GainVsTheta[360]
      elif t<=30:
        value = GainVsTheta[30]
      elif 30<t<=60:
        value = GainVsTheta[60]
      elif 60<t<=90:
        value = GainVsTheta[90]
      elif 90<t<=120:
        value = GainVsTheta[120]
      elif 120<t<=150:
        value = GainVsTheta[150]
      elif 150<t<=180:
        value = GainVsTheta[180]
      elif 180<t<=210:
        value = GainVsTheta[210]
      elif 210<t<=240:
        value = GainVsTheta[240]
      elif 240<t<=270:
        value = GainVsTheta[270]
      elif 270<t<=300:
        value = GainVsTheta[300]
      elif 300<t<=330:
        value = GainVsTheta[330]
      elif 330<t<=360:
        value = GainVsTheta[360]
      Gain.append(value-Path_Loss[i])
  Magnitude=[]
  for g in Gain:
    mag=pow(10,(g/10))
    Magnitude.append(mag)
  return Magnitude
#measureSNR(ant,SIZE_H,SIZE_W,typeOfAntenna="omnidirectionnel")


In [ ]:
measureSNR(ant,SIZE_H,SIZE_W,typeOfAntenna="omnidirectionnel")

In [ ]:
def calSNRlow(ant,SNR,TypeOfReward,TupleWallsIndex=TupleWallsIndex):
#counter =0
#for i in range(len(SNR)):
#    if int(SNR[i]) < snr_range["deadzone"]:
#      counter+=1
#      percent=counter*100/len(SNR)
    #return percent
  ## check if the antenna is inside the wall 
  if TypeOfReward=="AverageSNR":
    if (ant.x,ant.y) in TupleWallsIndex:
      return 0
    else:
      for i in range(SIZE_H):
        for j in range(SIZE_W):
          if (i,j) in TupleWallsIndex:
            SNR[i+(SIZE_W)*j]*=0
          else:
            continue
    return np.mean(SNR)
  else:
    counter =0
    for i in range(len(SNR)):
      if int(SNR[i]) < snr_range["deadzone"]:
        counter+=1
        percent=counter*100/len(SNR)
  return percent

#calSNRlow(ant,measureSNR(ant,SIZE_H,SIZE_W),TypeOfReward="AverageSNR",TupleWallsIndex)

In [ ]:
HM_EPISODES = 2500
epsilon = 0.5  # randomness
EPS_DECAY = 0.9999  # Every episode will be epsilon*EPS_DECAY
SHOW_EVERY = 100  # how often to play through env visually.

start_q_table = None  # if we have a pickled Q table, we'll put the filename of it here.

LEARNING_RATE = 0.1
DISCOUNT = 0.95

ANTENNA_N = 1  # NUMBER OF ANTENNAS 
#SNR RANGES
snr_range={"highSNR_max":-50,"highSNR_min":-80,
           "mediumSNR_max":-90,"mediumSNR_min":-100,
          "deadzone":-120}

In [ ]:
x = SIZE_W # Building X
y = SIZE_H # Building Y
z = 4 # actions

q_table = np.zeros((x, y, z))
# q_table *= 0
print(q_table.shape)


In [ ]:
def TrainingAgent(HM_EPISODES=HM_EPISODES,EPS_DECAY=EPS_DECAY,LEARNING_RATE=LEARNING_RATE,DISCOUNT=DISCOUNT,epsilon=epsilon,
                  SIZE_H=SIZE_H,SIZE_W=SIZE_W,typeOfAntenna="omnidirectionnel",TypeOfReward="AverageSNR",TupleWallsIndex=TupleWallsIndex):
  dict_values_edges=dict_Bet_Edges(TupleWallsIndex,SIZE_H,SIZE_W)
  episode_rewards = []
  reward=0
  for episode in range(HM_EPISODES):
      x_=np.random.randint(0, SIZE_H)
      y_=np.random.randint(0, SIZE_W)
      ant = antenna(x_,y_)
      if episode % SHOW_EVERY == 0:
          print(f"on #{episode}")
       # print(f"{SHOW_EVERY} ep mean: {np.mean(episode_rewards[-SHOW_EVERY:])}")
          show = True
      else:
          show = False

      episode_reward = 0
      for i in range(200):
          antennaPos = (ant) # Current Antenna X, Y Co-ord.
         # choose an action 
          if np.random.random() > epsilon: 
              action = np.argmax(q_table[ant.x][ant.y])
          else:
              action = np.random.randint(0, 4) #low is inclusive high is exclusive 
          #print(action)
        # Take the action!
          ant.action(action) # move the  antenna
        ###
        #calculate the SNR
          snr=measureSNR(ant,SIZE_H,SIZE_W,typeOfAntenna=typeOfAntenna,dict_values_edges=dict_values_edges)
          percent=calSNRlow(ant,snr,TypeOfReward=TypeOfReward,TupleWallsIndex=TupleWallsIndex)
        ##
          reward=percent
          ###
        #current_q = q_table[state][action]
          newAntennaPos = (ant)
          current_q = q_table[ant.x][ant.y][action] # Current State
          max_future_q = np.max(q_table[ant.x][ant.y]) # Max State
          new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
          q_table[ant.x][ant.y][action] = new_q
      

          #episode_reward += reward
      #print(episode)
        #print(episode_reward)
      #episode_rewards.append(episode_reward)
      epsilon *= EPS_DECAY
  return q_table

In [ ]:
q_table=TrainingAgent(HM_EPISODES=HM_EPISODES,EPS_DECAY=EPS_DECAY,LEARNING_RATE=LEARNING_RATE,DISCOUNT=DISCOUNT,epsilon=epsilon,
                  SIZE_H=SIZE_H,SIZE_W=SIZE_W,typeOfAntenna="omnidirectionnel",TypeOfReward="AverageSNR",TupleWallsIndex=TupleWallsIndex)

In [ ]:
def GetBestPosition (q_table): 
  xval=[]
  yval=[]
  max_value =np.max((q_table))
  print(max_value)
  for i in range(SIZE_H):
      for j in range(SIZE_W):
        for k in range(4):
          if max_value  == (q_table[i][j][k]):
              xval.append(i)
              yval.append(j)
          else :
              continue
  return xval,yval

  
xval,yval=GetBestPosition (q_table)
plt.scatter(xval, yval, color= "blue", linewidths =2 ,marker ="s")
plt.xlim(0, SIZE_H-1)
plt.ylim(0, SIZE_W-1)
plt.imshow(edges)
plt.show()
#print(q_table)
#print(xval)


In [ ]:
print(xval[0],yval[0])
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
# sns.set()

ant=antenna(xval[0],yval[0])
snr=measureSNR(ant,SIZE_H,SIZE_W,typeOfAntenna="omnidirectionnel")
# convert from linear to db
snr_db=[] 
for i in (snr):
  out_=10*np.log10(i)
  snr_db.append(out_)
print(snr_db)
# Build data
max_value_snr=np.max(snr_db)
min_value_snr=np.min(snr_db)
print(max_value_snr)
print(min_value_snr)

In [ ]:
## 1D heatmap
plt.figure()
plt.suptitle("Seaborn heatmap example to display 2d percentage values", color='m')
sns.set(font_scale=0.8)

print(len(snr_db))
snr_db=np.reshape(snr_db,((SIZE_H*SIZE_W-1),1)) ## size_w*size_h -1
ax = sns.heatmap(snr_db, vmin=min_value_snr, vmax=max_value_snr)
ax.set(xlabel='x label', ylabel='y label')

plt.show()

In [ ]:
rows=[]
row=[]
#print(type(snr_db[0]))
# insert a value
if SIZE_H>SIZE_W:
      position=(SIZE_W*(xval[0])) + (yval[0])
else:
      position=(SIZE_H*(yval[0])) + (xval[0])
#position=(SIZE_H*(yval[0]))+ xval[0]
snr_db_=np.insert(snr_db,position,[0])
print(snr_db_)
"""
for j in range(SIZE_W):
  for i in range (SIZE_H):
    value=snr_db_[i+(SIZE_W)*j].astype(int)
    row.append(value)
  rows.append(row)
  row=[]
print(rows)
for line in rows:
  print(line)
"""
rows=np.reshape(snr_db_,(1,SIZE_H*SIZE_W))

In [ ]:
#2D heatmap
plt.figure()
plt.suptitle("Seaborn heatmap example to display 2d percentage values", color='m')
sns.set(font_scale=0.8)

#print(type(rows))
rows_db=rows
rows_db=np.reshape(rows,(SIZE_W,SIZE_H))
ax = sns.heatmap(rows_db, vmin=min_value_snr, vmax=max_value_snr)
ax.set(xlabel='x label', ylabel='y label')


plt.show()

plt.scatter(xval, yval, color= "green", linewidths =6 ,marker ="s")
plt.xlim(0, SIZE_H-1)
plt.ylim(0, SIZE_W-1)
plt.imshow(edges)
plt.show()

loop everywhere and calculate average SNR s

In [ ]:
#looping on the area 
#brute force check  get the optimal decision 
AntTest=antenna(2,3)
EveryWhere={}
max=-200
#Move antenna Every where 
for i in range(SIZE_H):
  for j in range(SIZE_W):
    AntTest.x =i
    AntTest.y = j
    AvgSnr=calSNRlow(AntTest,measureSNR(AntTest,SIZE_H,SIZE_W,typeOfAntenna="omnidirectionnel"),TypeOfReward="AverageSNR")
    if AvgSnr>max:
      max=AvgSnr
    list_=[AntTest.x,AntTest.y,AvgSnr]
    EveryWhere[AvgSnr]=[AntTest.x,AntTest.y]
print(max,EveryWhere[max])
print(EveryWhere)
## x,y,meanSNR   